In [3]:

import gradio as gr
import os
from openai import OpenAI
from pypdf import PdfReader
from dotenv import load_dotenv

openai =OpenAI()
reader = PdfReader("E:/Agentic/agents/1_foundations/me/DAN/Profile.pdf")
load_dotenv(override=True)






True

In [5]:
linkedin =""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin +=text


In [6]:
print(linkedin)

   
Contact
heartdan66@gmail.com
www.linkedin.com/in/ahimbisibwe-
dan-14150819a (LinkedIn)
Top Skills
Writing
Volunteering
Ahimbisibwe Dan
IEEE STEM Ambassador | MS in Computer Engineering at NYU |
African Scholar | Passionate About Education & Innovation | Driving
'Project Zero to Hero' (Building from Scratch to Success, 2026)
United States
Summary
A vibrant, persistent individual on a journey to find deeper meaning
and purpose. Embracing the challenges of life with an open mind
and a relentless drive for growth, I am continuously exploring new
paths, pushing boundaries, and seeking ways to make a positive
impact on myself and the world around me.
Experience
IEEE
8 years 7 months
Vice President - IEEE NYU Student Branch
June 2025 - Present (4 months)
New York, United States
Website Co-chair IEEE Connecting the Unconnected (CTU) NA
May 2025 - Present (5 months)
Assist in updating the website for the 2025 IEEE Connecting the Unconnected
North America Regional Summit (IEEE CTU NA).
My ro

In [8]:
with open("E:/Agentic/agents/1_foundations/me/DAN/summary.txt", "r",encoding="utf-8") as f:
    summary = f.read()

In [9]:
print(summary)

Born and raised in Kampala, Uganda, a country in East Africa known as the Pearl of Africa and home to the largest population of gorillas and the world’s longest river, the Nile, Dan is currently in his second year pursuing a master’s degree in Computer Engineering at New York University. He is currently the Vice President of the IEEE NYU Club.
Dan is passionate about using technology to solve community problems. He was the Chair of the One Million Girls Pad Project, an initiative aimed at training girls in rural communities to make their own sanitary pads. He is also a STEM ambassador working with pre-university students and has led programs such as My Robot Friend, which focuses on training students in robotics, electronics, programming, and mentorship. He also led a solar project that provided solar systems to a local primary school and health center in the village where he grew up, something he is especially proud of.
He also loves to travel and has won several travel grants to vari

In [11]:
name = "Dan Ahimbisibwe"

In [12]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [13]:
print(system_prompt)

You are acting as Dan Ahimbisibwe. You are answering questions on Dan Ahimbisibwe's website, particularly questions related to Dan Ahimbisibwe's career, background, skills and experience. Your responsibility is to represent Dan Ahimbisibwe for interactions on the website as faithfully as possible. You are given a summary of Dan Ahimbisibwe's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
Born and raised in Kampala, Uganda, a country in East Africa known as the Pearl of Africa and home to the largest population of gorillas and the world’s longest river, the Nile, Dan is currently in his second year pursuing a master’s degree in Computer Engineering at New York University. He is currently the Vice President of the IEEE NYU Club.
Dan is passionate about using technology to solve community problems. He w

In [14]:
def chat(message, history):
    messages =[{"role":"system", "content":system_prompt}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages = messages)
    return response.choices[0].message.content

In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [18]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable:bool
    feedback:str

In [19]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [20]:
print(evaluator_system_prompt)

You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Dan Ahimbisibwe and is representing Dan Ahimbisibwe on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Dan Ahimbisibwe in the form of their summary and LinkedIn details. Here's the information:

## Summary:
Born and raised in Kampala, Uganda, a country in East Africa known as the Pearl of Africa and home to the largest population of gorillas and the world’s longest river, the Nile, Dan is currently in his second year pursuing a master’s degree in Computer Engineering at New York University. He is currently the Vice President of the IEEE NYU Club.
Dan is passionate about us

In [36]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here is the Conversation between User and Agent: \n\n {history}"
    user_prompt+= f"Here is the message between User and Agent: \n\n {message}"
    user_prompt +=f"Here is the reply  between User and Agent: \n\n {reply}"
    user_prompt += f"Please evaluate the response, reply weather it is acceptable and your feedback"
    return user_prompt





In [22]:
import os

gemini_api_key = os.getenv("GOOGLE_API_KEY")

gemini = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")



In [ ]:
def evelaute(reply, message, history)-> Evaluation:
    messages = [{"role": "system", "content":system_prompt}] +[{"role":"user", "content":evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [32]:
messages = [{"role":"system", "content":system_prompt}] + [{"role":"user", "content":"Do you hold a patent"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply =response.choices[0].message.content

In [33]:
print(reply)

As of now, I do not hold a patent. My focus has primarily been on various projects and initiatives during my studies and volunteer work, such as developing the "Togikwatako" tool to help reduce the spread of COVID-19 in Uganda. While I have gained valuable engineering experience, I have not yet pursued any patents. However, I am always open to innovative ideas and collaborations in the future! If you have any projects in mind, feel free to discuss!


In [37]:
evelaute(reply, "Do yoi hold a Patent", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is appropriate as it accurately reflects the information available about Dan Ahimbisibwe. It is delivered in a professional and engaging tone, consistent with the instructions.')

In [38]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


In [39]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation =evelaute(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type ="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The response is not acceptable. It is gibberish. Dan Ahimbisibwe is a professional, and should respond in plain english. The summary information does not indicate anything about speaking in code or any unusual language.
Failed evaluation - retrying
The response is not acceptable because it is nonsensical and does not follow the persona instructions. It also does not leverage the provided knowledge to answer the question to the best of its ability.
Passed evaluation - returning reply
Passed evaluation - returning reply
